In [130]:
# timing
from time import perf_counter
# used for saving datas
import time
import os

In [131]:
# numpy provides arrays and useful functions for working with them
import numpy

import cupy
import cupyx.scipy.ndimage

In [132]:
class neuralNetwork:
    """neural network class definition"""
    
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        """initialise the neural network"""
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node i to j in the next layer
        # w11 w21
        # w12 w22 etc
        self.wih = cupy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = cupy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))
        
        #learning rate
        self.lr = learningrate
        
        #activation function is the sigmoid function
        self.activation_function = lambda x: 1 / (1 + cupy.exp(x) ** (-1))
        
        pass
    
    
    def train(self, inputs_list, targets_list):
        """train the neural network"""
        # convert inputs list to 2d array
        inputs = cupy.array(inputs_list, ndmin=2).T
        targets = cupy.array(targets_list, ndmin=2).T

        # calculate signals into hidden layer
        hidden_inputs = cupy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calcute signals into final output layer
        final_inputs = cupy.dot(self.who, hidden_outputs)
        # calculate signals into final output layer
        final_outputs = self.activation_function(final_inputs)

        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = cupy.dot(self.who.T, output_errors)

        # update the weights for the links between the hidden and output layers
        # S' = S(1 - S)
        # dE / Dw = K * e * o * (1 - o)
        self.who += self.lr * cupy.dot((output_errors * final_outputs * (1.0 - final_outputs)), cupy.transpose(hidden_outputs))
        self.wih += self.lr * cupy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), cupy.transpose(inputs))

        pass


    def query(self, inputs_list):
        """query the neural network"""
        # convert inputs list to 2d array
        inputs = cupy.array(inputs_list, ndmin=2).T

        # calculate signals into hidden layer
        hidden_inputs = cupy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)

        # calcute signals into final output layer
        final_inputs = cupy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

In [133]:
def save_w(wih, who, hnodes=400):
    print("Saving Data...")
    
    t = time.asctime(time.localtime(time.time()))
    t = t.replace(':', '_')

    filepath = "saved_data/" + t + "/"
    os.mkdir(filepath)
    filename_wih = f"hn{hnodes}_wih_epoch_{e + 1}.csv"
    filename_who = f"hn{hnodes}_who_epoch_{e + 1}.csv"
    numpy.savetxt(filepath + filename_wih, wih)
    numpy.savetxt(filepath + filename_who, who)
    print("Saving Succeeded!")
    pass


In [134]:
# start counting
start = perf_counter()

In [135]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 400
output_nodes = 10

# learning rate
learning_rate = 0.01

# create instance of neural network
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [136]:
# load the mnist training data CSV file into a list
train_path = "F:/Documents/mnist_datasets/mnist_train.csv"
training_data_file = open(train_path, 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [137]:
# train the neural network

print("Training...")

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    print("Epoch:", e)
    # go through all records in the training data set
    # count = 1
    for record in training_data_list:
        # print(f"\rRound: {count}", end="")
        # count += 1
        # split the tecord by the ',' commas
        all_values = record.split(',')
        temp = list(map(eval, all_values[1:]))
        # scale and shift the inputs
        inputs = (cupy.asarray(temp) / 255 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired lavel which is 0.99)
        targets = cupy.zeros(output_nodes) + 0.01
        # all_values[0] is the target lavel for this record
        targets[int(all_values[0])] = 0.99
        # print(type(inputs))
        # print(type(targets))
        # input("")
        n.train(inputs, targets)
        
        # create rotated variations
        # rotated anticlockwise by x degrees
        
        inputs_plusx_img = cupyx.scipy.ndimage.interpolation.rotate(inputs.reshape(28, 28), 10, cval=0.01, order=1, reshape=False)
        n.train(inputs_plusx_img.reshape(784), targets)
        # rotated clockwise by x degrees
        inputs_minusx_img = cupyx.scipy.ndimage.interpolation.rotate(inputs.reshape(28, 28), -10, cval=0.01, order=1, reshape=False)
        n.train(inputs_minusx_img.reshape(784), targets)
        
    # print("")


Training...
Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4


In [138]:
# load the mnist test data CSV file into a list
test_path = "F:/Documents/mnist_datasets/mnist_test.csv"
test_data_file = open(test_path, 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [139]:
# test the neural network

print("Testing...")

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    temp = list(map(eval, all_values[1:]))
    inputs = (cupy.asarray(temp) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = cupy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)


Testing...


In [140]:
save_w(cupy.asnumpy(n.wih), cupy.asnumpy(n.who), hidden_nodes)

Saving Data...
Saving Succeeded!


In [141]:
# calculate the performance score, the fraction of correct answers
scorecard_array = cupy.asarray(scorecard)
print("performance =", scorecard_array.sum() / scorecard_array.size)

performance = 0.9717


In [143]:
# stop counting
end = perf_counter()
dur = end - start
mins = (end - start) // 60
secs = dur % 60
print("I'v spent {:.00f} mins {:.00f} secs on this.".format(mins, secs))

I'v spent 42 mins 21 secs on this.
